In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV
from sklearn.metrics import SCORERS, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import plotly.express as px
import plotly.graph_objects as go
from catboost import CatBoostRegressor, Pool

import numpy as np
from math import sqrt
pd.set_option("display.max_columns", 999)

# ML Regression techniques

## Regression problem

#### History :)
The term "regression" was coined by Francis Galton in the nineteenth century to describe a biological phenomenon. The phenomenon was that the heights of descendants of tall ancestors tend to regress down towards a normal average (a phenomenon also known as regression toward the mean).

#### Definition
Regression predictive modeling is the task of approximating a mapping function from input variables to a numerical output variable.

#### Most common regression models
* Linear Regression
    * Lasso
    * Ridge
    * ElasticNet
* Generalized Linear Regression
    * Logistic
    * Poisson
* Decision Trees
* Ensemble methods
    * Random Forests
    * Gradient Boosting
* Neural Networks
* Bayesian Regression
* SVM

# High Variance vs. High Bias problem

## High Bias
Bias is the algorithm’s tendency to consistently learn the wrong thing by not taking into account all the information in the data (underfitting)

#### Proposals
* Try more complex model.
* Add features.

## High Variance
Variance is the algorithm’s tendency to learn random things irrespective of the real signal by fitting highly flexible models that follow the error/noise in the data too closely (overfitting).

#### Proposals
* Try simpler model.
* Remove / reduce some features.
* Add regularization parameters.


<img src="img/Bias-Variance-Tradeoff-In-Machine-Learning.png" style="height:600px">

## ElasticNet Model a.k.a. From LM to ML

### Motivation
When we have lots of features, we want to be able to penalize their size and number and thus balance the Bias/Variance problem. 

Ideally we want to do this by some neat parametrization.


### Regularized loss function !

$L(\hat{y}, y)_{reg} = L(\hat{y}, y) + \lambda * R(\beta)$.

##### The loss function for ElasticNet regression is
$L(\hat{y}, y)_{lasso} = \sum_{p}{(\hat{y}-y)^2} + \lambda * \big[(1-\alpha)/2\sum_{p}{|\beta|} + \alpha \sum_{p}{\beta ^2}\big]$, where $\lambda \in N, \alpha \in [0, 1]$.

<img src="img/lasso_vs_ridge.png" />

## But how is $\lambda$ set? => CrossValidation!

<img src="img/cv_mse.png" />

# Back to the code

In [15]:
# define Root MSE function
def rmse(x, y):
    return sqrt(mean_squared_error(x, y))

In [4]:
# load the dataset into a Pandas dataframe
df = pd.read_csv('data/attrition.csv')

# drop columns with 
df.drop(['EmployeeNumber', 'Attrition', 'Over18', 'StandardHours', 'EmployeeCount'], axis=1, inplace=True)

In [5]:
# what columns fo we have
df.columns

Index(['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [8]:
# lets have a look at a scatter matrix plot
inspect_cols = ['MonthlyIncome','EducationField',
       'JobLevel', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']

# fig = px.scatter_matrix(df[inspect_cols])
# # increase the resolution
# fig.update_layout(
#     autosize=False,
#     width=2*1024,
#     height=2*720,
#     paper_bgcolor="LightSteelBlue",
# )
# #plot the scatter matrix plot
# fig.show()

In [9]:
# lets make it some fun
df.drop('JobLevel', axis=1, inplace=True)

KeyError: "['JobLevel'] not found in axis"

In [10]:
# select response variable and features
target_col_name = 'MonthlyIncome'
num_feature_cols = [
        'Age', 'DailyRate','DistanceFromHome', 'Education',
        'HourlyRate', 'EnvironmentSatisfaction', 'JobInvolvement',
        'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike',
        'RelationshipSatisfaction', 'StockOptionLevel', 'PerformanceRating',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
        'YearsWithCurrManager', 'MonthlyRate']
cat_feature_cols = [x for x in df.columns if x not in num_feature_cols and x not in [target_col_name]]

In [11]:
# cast numerical columns as float
for col in num_feature_cols:
    df[col] = df[col].astype(float)

In [12]:
# create target array and numeric features dataframe
df_target = np.ravel(df[[target_col_name]])
df_features = df[num_feature_cols]

In [29]:
# split the dataframe to train and test parts
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.3, random_state=1)

In [30]:
# fit simple linear regression
m0 = LinearRegression().fit(X_train, y_train)
m0_rmse = rmse(m0.predict(X_test), y_test)

print(f"Model rmse: {m0_rmse} \n")
print(f"Model coefs: {[int(x) for x in m0.coef_]}")

Model rmse: 2986.4758949208863 

Model coefs: [-20, 0, -7, 64, -2, -118, 19, 0, -29, 41, 36, 85, -810, 465, -49, 137, 82, -54, 33, -81, 0]


In [31]:
# lets try elastic net
m1 = ElasticNet(alpha=1, l1_ratio=0.5).fit(X_train, y_train)
m1_rmse = rmse(m1.predict(X_test), y_test)
hyper_pars = {'alpha': m1.alpha, 'l1_ratio': m1.l1_ratio}


print(f"Model rmse: {m1_rmse} \n")
print(f"Model coefs: {[int(x) for x in m1.coef_]} \n")
print(f"Model hyper_params: {hyper_pars}")

Model rmse: 2972.4704466171906 

Model coefs: [-14, 0, -7, 47, -2, -80, 9, 1, -21, -12, 23, 47, -77, 454, -36, 66, 81, -49, 27, -71, 0] 

Model hyper_params: {'alpha': 1, 'l1_ratio': 0.5}


# Cross validation - hyperparameter tunning

### K-Fold CV

<img src="img/kfold.png" />

### GridSearch CV Workflow

<img src="img/grid_search_workflow.png" />

In [32]:
# gridsearch through l1_ratio and alpha parameters grid
param_grid = {'l1_ratio': [.1, .5, .7, .9, .95, 1], 'alpha':[x**2 for x in range(30) if x > 0]}
m2 = GridSearchCV(estimator=ElasticNet(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m2_mse = rmse(m2.predict(X_test), y_test)

print(f"Model mse: {m2_mse} \n")
print(f"Model coefs: {[int(x) for x in m2.best_estimator_.coef_]} \n")
print(f"Model hyper_params: {m2.best_params_}")

Model mse: 2988.27183682534 

Model coefs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 441, 0, 0, 16, 0, 0, 0, 0] 

Model hyper_params: {'alpha': 841, 'l1_ratio': 1}


In [33]:
# printout coefs with their names
dict(zip(X_train.columns, m2.best_estimator_.coef_))

{'Age': -0.0,
 'DailyRate': -0.050895142340134335,
 'DistanceFromHome': -0.0,
 'Education': 0.0,
 'HourlyRate': -0.0,
 'EnvironmentSatisfaction': -0.0,
 'JobInvolvement': -0.0,
 'JobSatisfaction': 0.0,
 'NumCompaniesWorked': -0.0,
 'PercentSalaryHike': -0.0,
 'RelationshipSatisfaction': 0.0,
 'StockOptionLevel': 0.0,
 'PerformanceRating': -0.0,
 'TotalWorkingYears': 441.0828861678296,
 'TrainingTimesLastYear': -0.0,
 'WorkLifeBalance': 0.0,
 'YearsAtCompany': 16.007755120757103,
 'YearsInCurrentRole': -0.0,
 'YearsSinceLastPromotion': 0.0,
 'YearsWithCurrManager': -0.0,
 'MonthlyRate': 0.023338738149699174}

## Transformers & Pipelines

### a.k.a. how to do some feture engeneering and not go crazy through CV

<img src="img/pipeline-diagram.png" />

**Note: The pipeline object behaves just like any other estimator object. Which is cool.**

#### OK, lets add some categorical features!

In [36]:
# create target array and features dataframe
add_cat_features = cat_feature_cols
df_target = np.ravel(df[[target_col_name]])
df_features_all = df[num_feature_cols + add_cat_features]

# split the dataframe to train and test parts
X_train, X_test, y_train, y_test = train_test_split(df_features_all, df_target, test_size=0.3, random_state=1)

In [37]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(), add_cat_features)], remainder='passthrough')
estimators = [('column_trans', column_trans), ('reg', LinearRegression())]

pipe0 = Pipeline(estimators)
pipe0.fit(X_train, y_train)

pipe0_mse = rmse(pipe0.predict(X_test), y_test)

print(f"Model mse: {pipe0_mse} \n")
print(f"Model coefs: {[int(x) for x in pipe0.named_steps['reg'].coef_]} \n")

Model mse: 1672.2849470373058 

Model coefs: [-78, 31, 46, -384, 490, -105, 451, -97, 86, -212, -157, -69, -31, 31, -555, -2052, -3609, 6904, -460, 6617, -3590, -185, -3068, 26, -72, 45, -9, 9, -6, 0, 4, -40, 0, -34, -121, 45, 0, 28, 32, 31, -597, 202, -47, 112, 45, -33, 39, -46, 0] 



In [38]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder='passthrough')
estimators = [('column_trans', column_trans), ('reg', ElasticNet())]

pipe1 = Pipeline(estimators)
pipe1.fit(X_train, y_train)

pipe1_mse = rmse(pipe1.predict(X_test), y_test)
hyper_pars = {'alpha': pipe1.named_steps['reg'].alpha, 'l1_ratio': pipe1.named_steps['reg'].l1_ratio}

print(f"Model mse: {pipe1_mse} \n")
print(f"Model coefs: {[int(x) for x in pipe1.named_steps['reg'].coef_]} \n")
print(f"Model hyper_params: {hyper_pars}")

Model mse: 2777.576814341133 

Model coefs: [24, -41, 15, -21, -64, 87, 7, 26, 35, -47, 15, -39, 22, -22, -9, -61, -466, 487, 23, 575, -487, 56, -117, 54, 19, -74, 16, -16, -15, 0, -6, 38, -1, -71, 11, 10, -17, -13, 23, 18, -72, 431, -33, 54, 79, -50, 28, -71, 0] 

Model hyper_params: {'alpha': 1.0, 'l1_ratio': 0.5}


In [39]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder='passthrough')
estimators = [('column_trans', column_trans), ('reg', ElasticNet())]
pipe1 = Pipeline(estimators)
param_grid = {'reg__l1_ratio': [.1, .5, .7, .9, .95, 1], 'reg__alpha':[0.2, 0.5, 1, 4, 10, 20, 40, 100]}
m3 = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m3_mse = rmse(m3.predict(X_test), y_test)

print(f"Model mse: {m3_mse} \n")
print(f"Model coefs: {[int(x) for x in m3.best_estimator_.named_steps['reg'].coef_]} \n")
print(f"Model hyper_params: {m3.best_params_}")

Model mse: 1677.0303359673717 

Model coefs: [0, 0, 0, -103, 239, 0, 0, 0, 54, -85, 0, 0, -3, 0, 0, -1448, -2941, 7001, 0, 6919, -2910, 53, -2516, 0, -55, 0, 0, 0, -6, 0, 4, -19, 0, -24, -97, 37, 0, 12, 21, 9, -406, 213, -39, 82, 44, -30, 35, -45, 0] 

Model hyper_params: {'reg__alpha': 10, 'reg__l1_ratio': 1}


In [40]:
# added scaler
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder=MinMaxScaler())
estimators = [('column_trans', column_trans), ('reg', ElasticNet())]
pipe1 = Pipeline(estimators)
param_grid = {'reg__l1_ratio': [.1, .5, .7, .9, .95, 1], 'reg__alpha':[0.2, 0.5, 1, 4, 10, 20, 40, 100]}
m4 = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m4_mse = rmse(m4.predict(X_test), y_test)

print(f"Model mse: {m4_mse} \n")
print(f"Model coefs: {[int(x) for x in m4.best_estimator_.named_steps['reg'].coef_]} \n")
print(f"Model hyper_params: {m4.best_params_}")

Model mse: 1671.9526541529583 

Model coefs: [0, 0, 0, 0, 115, 0, 0, 0, 0, -31, 0, 0, 0, 0, 0, -1336, -2870, 7041, 0, 6900, -2832, 0, -2390, 0, -18, 0, 0, 0, 0, 0, 0, 0, 0, 0, -74, 8, 0, 0, 0, 0, -228, 8143, 0, 0, 90, 0, 255, 0, 0] 

Model hyper_params: {'reg__alpha': 20, 'reg__l1_ratio': 1}


## Decision trees - bricks of ensemble models

#### What is it?
A model that consists of k nodes with binary decision rule. 

#### How does it predict?
1] Is your weight >= 90 -> True

2] Is your level of excersise >= 0 -> False

3] Do you have stressful job -> True

--> You will die at np.mean([63, 82, 54, 61, 64]) = 64.8

#### How is it trained?

1] Fow each candidate split $\theta$ (across all fetures), at node with $Q$ remaining data points, scoring function $G(Q, \theta)$ is computed as:

$G(Q, \theta) = \frac{n_{left}}{N_m} H(Q_{left}(\theta)) + \frac{n_{right}}{N_m} H(Q_{right}(\theta))$ 

Where $Q_{left}$ represents the left side of the node. The inpurity function $H()$ is defined as MSE for regression task.

2] You select $\theta^* = argmin_{\theta} G(Q, \theta)$ as your next split.

This is repeated until there is nothing to split or stopping criterion is hit.

#### Why is it so bad?
The decision trees tend to overfit.

<img src="img/tree_fit.png"/>

In [63]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder=MinMaxScaler())
estimators = [('column_trans', column_trans), ('reg', DecisionTreeRegressor())]
pipe2 = Pipeline(estimators)
param_grid = {'reg__max_depth': [3, 5]}
m4 = GridSearchCV(estimator=pipe2, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m4_mse = rmse(m4.predict(X_test), y_test)

print(f"Model mse: {m4_mse} \n")
print(f"Model hyper_params: {m4.best_params_}")

Model mse: 1780.9897028884568 

Model hyper_params: {'reg__max_depth': 5}


In [64]:
import graphviz 
dot_data = export_graphviz(m4.best_estimator_.named_steps['reg'], out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("img/tree_example")

'img/tree_example.pdf'

<img src="img/tree_example.pdf"/>

## Ensemble models - Random Forests

<img src="img/rf.png"/>

In [65]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder=MinMaxScaler())
estimators = [('column_trans', column_trans), ('reg', RandomForestRegressor())]
pipe2 = Pipeline(estimators)
param_grid = {'reg__max_depth': [5, 10, 15], 'reg__n_estimators':[40, 100, 150], 'reg__max_features':['auto', 'sqrt']}
m4 = GridSearchCV(estimator=pipe2, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m4_mse = rmse(m4.predict(X_test), y_test)

print(f"Model mse: {m4_mse} \n")
print(f"Model hyper_params: {m4.best_params_}")

Model mse: 1558.9378699145716 

Model hyper_params: {'reg__max_depth': 10, 'reg__max_features': 'auto', 'reg__n_estimators': 100}


## Ensemble models - Gradient Boosting

#### Weak learner -> errors -> Weak learner -> errors -> ...

If we define weak learner in $m$-th iteration step as $h_m$ and final model in step $m$ as $F_m$, then the iteration formula is:

$F_m(X) = F_{m-1}(X) + \lambda h_m(X)$,

where $X$ are the data and $\lambda$ is learning raste coefficient. Initial model $F_0$ is just weak learner on the data $X$.

<img src="img/xgb.png"/>

In [ ]:
column_trans = ColumnTransformer([('onehot', OneHotEncoder(dtype='int'), add_cat_features)], remainder=MinMaxScaler())
estimators = [('column_trans', column_trans), ('reg', GradientBoostingRegressor())]
pipe2 = Pipeline(estimators)
param_grid = {'reg__max_features': ['auto', 'sqrt'], 'reg__subsample': [0.1, 0.05, 0.4], 'reg__min_samples_leaf': [0.0025, 0.005, 0.01, 0.05, 0.1], 'reg__n_estimators':[30, 40, 50, 70, 200]}
m4 = GridSearchCV(estimator=pipe2, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', iid=False).fit(X_train, y_train)

m4_mse = rmse(m4.predict(X_test), y_test)

print(f"Model mse: {m4_mse} \n")
print(f"Model hyper_params: {m4.best_params_}")

...

# CatBoost

In [18]:
# initialize data
train_data = np.random.randint(0, 
                               100, 
                               size=(100, 10))
train_label = np.random.randint(0, 
                                1000, 
                                size=(100))
test_data = np.random.randint(0, 
                              100, 
                              size=(50, 10))
# initialize Pool
train_pool = Pool(train_data, 
                  train_label, 
                  cat_features=[0,2,5])
test_pool = Pool(test_data, 
                 cat_features=[0,2,5]) 

# specify the training parameters 
model = CatBoostRegressor(iterations=2, 
                          depth=2, 
                          learning_rate=1, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)
print(preds)

0:	learn: 282.7339829	total: 50.2ms	remaining: 50.2ms
1:	learn: 282.1288038	total: 52.7ms	remaining: 0us
[446.89976575 546.90499444 585.95009165 546.90499444 638.51599298
 546.90499444 591.87394552 446.89976575 446.89976575 585.95009165
 446.89976575 446.89976575 446.89976575 638.51599298 546.90499444
 446.89976575 471.55549188 638.51599298 446.89976575 546.90499444
 446.89976575 446.89976575 546.90499444 546.90499444 585.95009165
 446.89976575 446.89976575 471.55549188 585.95009165 446.89976575
 446.89976575 446.89976575 546.90499444 546.90499444 638.51599298
 446.89976575 485.94486295 491.86871682 446.89976575 446.89976575
 446.89976575 546.90499444 471.55549188 446.89976575 638.51599298
 585.95009165 446.89976575 446.89976575 446.89976575 446.89976575]


In [19]:
cat_feature_cols

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [25]:
# initialize Pool
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=cat_feature_cols)
test_pool = Pool(X_test, 
                 cat_features=cat_feature_cols) 

# specify the training parameters 
model = CatBoostRegressor(iterations=2, 
                          depth=2, 
                          learning_rate=1, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool)


m4_mse = rmse(model.predict(test_pool), y_test)
m4_mse

0:	learn: 2855.9589244	total: 3.88ms	remaining: 3.88ms
1:	learn: 2184.4195187	total: 6.12ms	remaining: 0us


2043.2030074055815

# Sources:
[Bias/Variance problem](https://www.learnopencv.com/bias-variance-tradeoff-in-machine-learning/)

[Scikit supervized models](https://scikit-learn.org/stable/supervised_learning.html)

[Bayessian regression post](https://stats.stackexchange.com/questions/252577/bayes-regression-how-is-it-done-in-comparison-to-standard-regression/252608)